# Introduction

## 1.1 Background
Jollibee is the largest fast food restaurant chain in the Philippines, while very well known around Southeast Asia and Middle East, there is only one Jollibee franchise in Toronto Canada.

## 1.2 Business Problem
Jollibee Foods Corporation (JFC) looks to expand its footprint in Toronto, Canada. As of July 2020, the only Jollibee franchise is in Scarborough and the company is looking for a similar neighborhood where they can open another franchise.

# Data

## 2.1 Data Sources
The neighbourhoods alongside their respective postal codes and boroughs were scraped from [Wikipedia](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M). Geographical coordinates for each neighbourhood were extracted from [here](https://cocl.us/Geospatial_data). The city of Toronto has provided [neighborhood profile data](https://open.toronto.ca/dataset/neighbourhood-profiles/) through their own [Open Data Portal](https://open.toronto.ca/).

I will be utilizing the Foursquare API to obtain/explore which neighborhoods has the highest frequency of fast food chains.

## 2.2 Collection
### 2.2.1 Scraping Toronto, Canada Wikipedia Page

In [1]:
import urllib.request
from bs4 import BeautifulSoup


url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")

Looking at the Wikipedia page, there's only one table in it and its class attribute is wikitable sortable. Using that information and the find function of the BeautifulSoup package, we find the table xml

In [2]:
postal_code_table = soup.find('table', class_='wikitable sortable')

Knowing how HTML tables are written, I looped through each `tr` of the HTML table and use follow the instructions:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [3]:
import pandas as pd
import numpy as np

PostalCode = []
Borough = []
Neighborhood = []

for row in postal_code_table.findAll('tr')[1:]:
    cells = row.findAll('td')
    borough = cells[1].find(text=True).strip()
    
    if borough != 'Not assigned':
        PostalCode.append(cells[0].find(text=True).strip())
        Borough.append(borough)
        neighborhood = cells[2].find(text=True).strip()
        if neighborhood == 'Not assigned': 
            Neighborhood.append(borough)
        else:
            Neighborhood.append(neighborhood)
            

data = []
for i, N in enumerate(Neighborhood):
    b = Borough[i]
    p = PostalCode[i]
    for n in N.split(','):
        n = n.strip()
        if '-' in n:
            n.replace(' - ', '-')

        d = {
            'Borough': b,
            'PostalCode': p,
            'Neighborhood': n.strip()
        }

        data.append(d)
        
neighborhood_df = pd.DataFrame(data)
neighborhood_df.head()

,Borough,PostalCode,Neighborhood
0,North York,M3A,Parkwoods
1,North York,M4A,Victoria Village
2,Downtown Toronto,M5A,Regent Park
3,Downtown Toronto,M5A,Harbourfront
4,North York,M6A,Lawrence Manor


### 2.2.2 Load Population, Ethnic, and Income Data

Next, let's explore the neighborhood profile data provided by Toronto. Let's take a look at the `Category` column.

In [4]:
initial_census = pd.read_csv('neighbourhood-profiles.csv')
np.unique(initial_census['Category'].values)

array(['Aboriginal peoples', 'Education', 'Ethnic origin',
       'Families, households and marital status', 'Housing',
       'Immigration and citizenship', 'Income', 'Journey to work',
       'Labour', 'Language', 'Language of work', 'Mobility',
       'Neighbourhood Information', 'Population', 'Visible minority'],
      dtype=object)

The following the data is needed to average income of each neighborhood, the Filipino population of each neighborhood, and the total population of each neighborhood.

#### 2.2.2.1 Getting the Average Income of each Neighborhood

In [6]:
average_income_df = initial_census[(initial_census['Category'] == 'Income') & (initial_census['Characteristic'] == 'Total income: Average amount ($)')]
average_income_df.drop(['_id', 'Category', 'Topic', 'Data Source', 'City of Toronto'], axis=1, inplace=True)
average_income_df.set_index('Characteristic', inplace=True)
average_income_df = average_income_df.transpose()

for column in average_income_df.columns:
    average_income_df[column] = average_income_df[column].str.replace(',', '').astype('int')

average_income_df.columns = ['Avarage Income']
average_income_df.head()

/Users/cube/.local/share/virtualenvs/Coursera_Capstone-_E-QZ5IB/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Avarage Income
Agincourt North,30414
Agincourt South-Malvern West,31825
Alderwood,47709
Annex,112766
Banbury-Don Mills,67757


#### 2.2.2.2 Filipino Population of each Neighborhood

In [7]:
filipino_df = initial_census[(initial_census['Category'] == 'Ethnic origin') & (initial_census['Characteristic'] == ' Filipino')]
filipino_df.drop(['_id', 'Category', 'Topic', 'Data Source', 'City of Toronto'], axis=1, inplace=True)
filipino_df.set_index('Characteristic', inplace=True)
filipino_df = filipino_df.transpose()

for column in filipino_df.columns:
    filipino_df[column] = filipino_df[column].str.replace(',', '').astype('int')

filipino_df.columns = ['Filipino Population']
filipino_df.head()

/Users/cube/.local/share/virtualenvs/Coursera_Capstone-_E-QZ5IB/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Filipino Population
Agincourt North,1465
Agincourt South-Malvern West,1405
Alderwood,660
Annex,435
Banbury-Don Mills,1280


#### 2.2.2.3 Total Population of each Neighborhood

In [9]:
pop_df = initial_census[(initial_census['Category'] == 'Population') & (initial_census['Characteristic'] == 'Population, 2016')]
pop_df.drop(['_id', 'Category', 'Topic', 'Data Source', 'City of Toronto'], axis=1, inplace=True)
pop_df.set_index('Characteristic', inplace=True)
pop_df = pop_df.transpose()

for column in pop_df.columns:
    pop_df[column] = pop_df[column].str.replace(',', '').astype('int')

pop_df.columns = ['Total Population']
pop_df.head()

/Users/cube/.local/share/virtualenvs/Coursera_Capstone-_E-QZ5IB/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Total Population
Agincourt North,29113
Agincourt South-Malvern West,23757
Alderwood,12054
Annex,30526
Banbury-Don Mills,27695


Next, let's merge all three dataframes.

In [10]:
from functools import reduce


dfs = [pop_df, filipino_df, average_income_df]
df_census = reduce(lambda left,right: pd.merge(left, right, left_index=True, right_index=True), dfs)
df_census.reset_index(inplace=True)
df_census.rename({'index': 'Neighborhood'}, axis=1, inplace=True)
df_census.head()

,Neighborhood,Total Population,Filipino Population,Avarage Income
0,Agincourt North,29113,1465,30414
1,Agincourt South-Malvern West,23757,1405,31825
2,Alderwood,12054,660,47709
3,Annex,30526,435,112766
4,Banbury-Don Mills,27695,1280,67757


### 2.2.3 Load Geodata

In [11]:
geospatial_df = pd.read_csv('Geospatial_Coordinates.csv')
geospatial_df['PostalCode'] = geospatial_df['Postal Code']
geospatial_df.drop(['Postal Code'], axis=1, inplace=True)
geospatial_df.head()

,Latitude,Longitude,PostalCode
0,43.806686,-79.194353,M1B
1,43.784535,-79.160497,M1C
2,43.763573,-79.188711,M1E
3,43.770992,-79.216917,M1G
4,43.773136,-79.239476,M1H


### 2.2.4 Merging all data into a single database

In [12]:
df = pd.merge(neighborhood_df, geospatial_df, on='PostalCode').drop(['PostalCode', 'Borough'], axis=1)
df = pd.merge(df, df_census, on='Neighborhood', how='left').dropna(how='any', axis=0).reset_index(drop=True)
df.head()

,Neighborhood,Latitude,Longitude,Total Population,Filipino Population,Avarage Income
0,Victoria Village,43.725882,-79.315572,17510.0,1290.0,35786.0
1,Regent Park,43.654260,-79.360636,10803.0,305.0,34597.0
2,Malvern,43.806686,-79.194353,43794.0,5095.0,29573.0
3,Rouge,43.806686,-79.194353,46496.0,4365.0,39556.0
4,Highland Creek,43.784535,-79.160497,12494.0,880.0,40972.0


#### 2.2.4.1 Get the percentage of Filipino in the total population

In [13]:
df['Percentage of Filipino'] = df['Filipino Population'] / df['Total Population'] * 100
df.drop(['Total Population', 'Filipino Population'], axis=1, inplace=True)
df.head()

,Neighborhood,Latitude,Longitude,Avarage Income,Percentage of Filipino
0,Victoria Village,43.725882,-79.315572,35786.0,7.367219
1,Regent Park,43.654260,-79.360636,34597.0,2.823290
2,Malvern,43.806686,-79.194353,29573.0,11.634014
3,Rouge,43.806686,-79.194353,39556.0,9.387904
4,Highland Creek,43.784535,-79.160497,40972.0,7.043381


# Analysis
## 3.1 Visualizing using Folium

In [14]:
from geopy.geocoders import Nominatim
address = 'Toronto'
geolocator = Nominatim(user_agent='to_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("The geographical coordinates of Toronto are {}, {}".format(latitude, longitude))

The geographical coordinates of Toronto are 43.6534817, -79.3839347


In [16]:
import folium
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
  label = '{}'.format(neighbourhood)
  label = folium.Popup(label)
  folium.CircleMarker(
      [lat,lng],
      radius=8,
      color='blue',
      popup=label,
      fill_color='#3186cc',
      fill_opacity=0.7,
      fill=True

  ).add_to(map_Toronto)

map_Toronto